<a href="https://colab.research.google.com/github/HalgasAdrian/CS5230-Coursework/blob/main/HW2_KMEANS_and_Gaussian_Mixtures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HW2 KMNEANS and Gaussian Mixtures


Problem 1: KMeans Theory

Problem 2: KMeans on data

Using Euclidian distance or dot product similarity (choose one per dataset, you can try other similarity metrics).

A) run KMeans on the MNIST Dataset, try K=10

B) run KMeans on the FASHION Dataset, try K=10

C) run KMeans on the 20NG Dataset, try K=20

You can use a library for distance/similarity but you have to implement your own kmeans (EM steps, termination criteria etc).
For all three datasets, evaluate the KMeans objective for a higher K (for example double) or smaller K(for example half).
For all three datasets, evaluate external clustering performance using data labels and performance metrics Purity and Gini Index (see [A] book section 6.9.2).

D) run soft Kmeans for MNIST dataset, use K=10. You can try beta=0.1, beta=1, beta=10. Evaluate performance.

A) Run KMeans on the MNIST Dataset, try K = 10

In [1]:
import os

base_url = 'https://web.archive.org/web/20220331230320/http://yann.lecun.com/exdb/mnist/'
files = [
    'train-images-idx3-ubyte.gz',
    'train-labels-idx1-ubyte.gz',
    't10k-images-idx3-ubyte.gz',
    't10k-labels-idx1-ubyte.gz'
]

# Directory to store MNIST data
os.makedirs('/content/mnist', exist_ok=True)

# Download each file
for file in files:
    url = f"{base_url}{file}"
    output_path = f"/content/mnist/{file}"
    !wget -O {output_path} {url}

print("All MNIST files downloaded.")

--2025-02-03 21:38:49--  https://web.archive.org/web/20220331230320/http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving web.archive.org (web.archive.org)... 207.241.237.3
Connecting to web.archive.org (web.archive.org)|207.241.237.3|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://web.archive.org/web/20220331225332/http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz [following]
--2025-02-03 21:38:51--  https://web.archive.org/web/20220331225332/http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Reusing existing connection to web.archive.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘/content/mnist/train-images-idx3-ubyte.gz’

/content/mnist/trai 100%[===================>]   9.45M  45.3MB/s    in 0.2s    

2025-02-03 21:38:54 (45.3 MB/s) - ‘/content/mnist/train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2025-02-03 21:38:54--  https://web.archive

In [2]:
import gzip
import numpy as np

# Function to extract gzip files
def extract_gz(file_path, out_path):
    with gzip.open(file_path, 'rb') as f_in:
        with open(out_path, 'wb') as f_out:
            f_out.write(f_in.read())

# Extract MNIST files
mnist_dir = '/content/mnist'
for file in files:
    gz_path = os.path.join(mnist_dir, file)
    out_path = gz_path.replace('.gz', '')
    extract_gz(gz_path, out_path)

print("All MNIST files extracted.")

# Function to parse MNIST image files
def load_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        f.read(16)  # Skip header
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data.reshape(-1, 28, 28)

# Function to parse MNIST label files
def load_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        f.read(8)  # Skip header
        data = np.frombuffer(f.read(), dtype=np.uint8)
        return data

# Load training and test data
X_train_mnist = load_mnist_images('/content/mnist/train-images-idx3-ubyte')
y_train_mnist = load_mnist_labels('/content/mnist/train-labels-idx1-ubyte')
X_test_mnist = load_mnist_images('/content/mnist/t10k-images-idx3-ubyte')
y_test_mnist = load_mnist_labels('/content/mnist/t10k-labels-idx1-ubyte')

print("MNIST data successfully loaded.")

All MNIST files extracted.
MNIST data successfully loaded.


KMeans from scratch

In [3]:
import numpy as np

class KMeans:
    def __init__(self, n_clusters=10, max_iter=300, tol=1e-4):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.centroids = None
        self.labels = None

    def fit(self, X):
        n_samples, n_features = X.shape

        # Initialize centroids randomly
        random_indices = np.random.choice(n_samples, self.n_clusters, replace=False)
        self.centroids = X[random_indices]

        for i in range(self.max_iter):
            # Assign clusters
            distances = self._compute_distances(X)
            self.labels = np.argmin(distances, axis=1)

            # Update centroids
            new_centroids = np.array([X[self.labels == k].mean(axis=0) for k in range(self.n_clusters)])

            # Check for convergence
            if np.linalg.norm(new_centroids - self.centroids) < self.tol:
                break

            self.centroids = new_centroids

    def _compute_distances(self, X):
        distances = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            distances[:, k] = np.linalg.norm(X - self.centroids[k], axis=1)
        return distances

    def predict(self, X):
        distances = self._compute_distances(X)
        return np.argmin(distances, axis=1)

Kmeans on MNIST Dataset with K = 10

In [4]:
# Flatten the MNIST images
X_train_mnist_flat = X_train_mnist.reshape(X_train_mnist.shape[0], -1)

# Normalize the data
X_train_mnist_flat = X_train_mnist_flat / 255.0

# Run KMeans
kmeans_mnist = KMeans(n_clusters=10)
kmeans_mnist.fit(X_train_mnist_flat)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_mnist = kmeans_objective(X_train_mnist_flat, kmeans_mnist.centroids, kmeans_mnist.labels)
print(f"KMeans Objective for MNIST (K=10): {objective_mnist}")

KMeans Objective for MNIST (K=10): 2352828.0140824346


In [5]:
from sklearn.metrics import confusion_matrix

def purity_score(y_true, y_pred):
    confusion_mat = confusion_matrix(y_true, y_pred)
    return np.sum(np.amax(confusion_mat, axis=0)) / np.sum(confusion_mat)

def gini_index(y_true, y_pred):
    contingency = confusion_matrix(y_true, y_pred)
    cluster_sizes = np.sum(contingency, axis=0)

    # Handle empty clusters and division by zero
    valid_clusters = cluster_sizes > 0
    if np.sum(valid_clusters) == 0:
        return 1.0  # Return worst case when all clusters are empty

    # Calculate purities only for valid clusters
    cluster_purities = np.zeros_like(cluster_sizes, dtype=np.float64)
    cluster_purities[valid_clusters] = np.amax(contingency[:, valid_clusters], axis=0) / cluster_sizes[valid_clusters]

    # Calculate weighted Gini index (proper formula)
    total_samples = np.sum(cluster_sizes)
    weights = cluster_sizes[valid_clusters] / total_samples
    gini = 1 - np.sum(weights * (cluster_purities[valid_clusters] ** 2))

    return np.clip(gini, 0.0, 1.0)  # Ensure valid range

purity_mnist = purity_score(y_train_mnist, kmeans_mnist.labels)
gini_mnist = gini_index(y_train_mnist, kmeans_mnist.labels)

print(f"Purity for MNIST (K=10): {purity_mnist}")
print(f"Gini Index for MNIST (K=10): {gini_mnist}")

Purity for MNIST (K=10): 0.5906166666666667
Gini Index for MNIST (K=10): -3.5202164052678473


MNIST KMEANS K=20 with external clustering performance.

In [6]:
# Run KMeans
kmeans_mnist = KMeans(n_clusters=20)
kmeans_mnist.fit(X_train_mnist_flat)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_mnist = kmeans_objective(X_train_mnist_flat, kmeans_mnist.centroids, kmeans_mnist.labels)
print(f"KMeans Objective for MNIST (K=20): {objective_mnist}")

purity_mnist = purity_score(y_train_mnist, kmeans_mnist.labels)
gini_mnist = gini_index(y_train_mnist, kmeans_mnist.labels)

print(f"Purity for MNIST (K=20): {purity_mnist}")
print(f"Gini Index for MNIST (K=20): {gini_mnist}")

KMeans Objective for MNIST (K=20): 2111559.14306655
Purity for MNIST (K=20): 0.7052666666666667
Gini Index for MNIST (K=20): -10.586305611930403


MNIST KMEANS K=5 with external clustering performance.

In [7]:
# Run KMeans
kmeans_mnist = KMeans(n_clusters=5)
kmeans_mnist.fit(X_train_mnist_flat)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_mnist = kmeans_objective(X_train_mnist_flat, kmeans_mnist.centroids, kmeans_mnist.labels)
print(f"KMeans Objective for MNIST (K=5): {objective_mnist}")

purity_mnist = purity_score(y_train_mnist, kmeans_mnist.labels)
gini_mnist = gini_index(y_train_mnist, kmeans_mnist.labels)

print(f"Purity for MNIST (K=5): {purity_mnist}")
print(f"Gini Index for MNIST (K=5): {gini_mnist}")

KMeans Objective for MNIST (K=5): 2605960.404855432
Purity for MNIST (K=5): 0.45236666666666664
Gini Index for MNIST (K=5): nan


<ipython-input-5-e29d13acec4f>:10: RuntimeWarning: invalid value encountered in divide
  cluster_purities = np.amax(confusion_mat, axis=0) / cluster_sizes


B) run KMeans on the FASHION Dataset, try K=10

First we must load and prepare the data.

In [8]:
# Create directory
!mkdir -p /content/fashion-mnist

# Download files directly to the target directory
!wget -P /content/fashion-mnist https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-images-idx3-ubyte.gz
!wget -P /content/fashion-mnist https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-labels-idx1-ubyte.gz
!wget -P /content/fashion-mnist https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-images-idx3-ubyte.gz
!wget -P /content/fashion-mnist https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/t10k-labels-idx1-ubyte.gz

# Extract files using Python's gzip (no overwrite prompts)
import gzip
import shutil

for gz_file in ['train-images-idx3-ubyte.gz', 'train-labels-idx1-ubyte.gz',
                't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz']:
    with gzip.open(f'/content/fashion-mnist/{gz_file}', 'rb') as f_in:
        with open(f'/content/fashion-mnist/{gz_file[:-3]}', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

print("Files successfully downloaded and extracted!")

--2025-02-03 21:49:29--  https://github.com/zalandoresearch/fashion-mnist/raw/master/data/fashion/train-images-idx3-ubyte.gz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-images-idx3-ubyte.gz [following]
--2025-02-03 21:49:29--  https://raw.githubusercontent.com/zalandoresearch/fashion-mnist/master/data/fashion/train-images-idx3-ubyte.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26421880 (25M) [application/octet-stream]
Saving to: ‘/content/fashion-mnist/train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]  25

In [9]:
import numpy as np
import os

def load_fashion(path, kind='train'):
    """Load Fashion MNIST data from path"""
    labels_path = os.path.join(path, f'{kind}-labels-idx1-ubyte')
    images_path = os.path.join(path, f'{kind}-images-idx3-ubyte')

    with open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

    with open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                             offset=16).reshape(len(labels), 784)

    return images.reshape(-1, 28, 28), labels

# Load the data
try:
    X_train_fashion, y_train_fashion = load_fashion('/content/fashion-mnist', kind='train')
    X_test_fashion, y_test_fashion = load_fashion('/content/fashion-mnist', kind='t10k')
    print("Data loaded successfully!")
except Exception as e:
    print(f"Error: {e}")

Data loaded successfully!


Now with our data loaded we can run KMEANS on the FASHION dataset.

In [10]:
# Flatten the Fashion images
X_train_fashion_flat = X_train_fashion.reshape(X_train_fashion.shape[0], -1)

# Normalize the data
X_train_fashion_flat = X_train_fashion_flat / 255.0

# Run KMeans
kmeans_fashion = KMeans(n_clusters=10)
kmeans_fashion.fit(X_train_fashion_flat)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_fashion = kmeans_objective(X_train_fashion_flat, kmeans_fashion.centroids, kmeans_fashion.labels)
print(f"KMeans Objective for FASHION (K=10): {objective_fashion}")

KMeans Objective for FASHION (K=10): 1906653.2385860477


In [11]:
from sklearn.metrics import confusion_matrix

def purity_score(y_true, y_pred):
    confusion_mat = confusion_matrix(y_true, y_pred)
    return np.sum(np.amax(confusion_mat, axis=0)) / np.sum(confusion_mat)

def gini_index(y_true, y_pred):
    contingency = confusion_matrix(y_true, y_pred)
    cluster_sizes = np.sum(contingency, axis=0)

    # Handle empty clusters and division by zero
    valid_clusters = cluster_sizes > 0
    if np.sum(valid_clusters) == 0:
        return 1.0  # Return worst case when all clusters are empty

    # Calculate purities only for valid clusters
    cluster_purities = np.zeros_like(cluster_sizes, dtype=np.float64)
    cluster_purities[valid_clusters] = np.amax(contingency[:, valid_clusters], axis=0) / cluster_sizes[valid_clusters]

    # Calculate weighted Gini index (proper formula)
    total_samples = np.sum(cluster_sizes)
    weights = cluster_sizes[valid_clusters] / total_samples
    gini = 1 - np.sum(weights * (cluster_purities[valid_clusters] ** 2))

    return np.clip(gini, 0.0, 1.0)  # Ensure valid range

purity_fashion = purity_score(y_train_fashion, kmeans_fashion.labels)
gini_fashion = gini_index(y_train_fashion, kmeans_fashion.labels)

print(f"Purity for MNIST (K=10): {purity_fashion}")
print(f"Gini Index for MNIST (K=10): {gini_fashion}")

Purity for MNIST (K=10): 0.5535666666666667
Gini Index for MNIST (K=10): -3.6140723217252573


FASHION KMEANS K=20 with external clustering performance.

In [12]:
# Run KMeans
kmeans_fashion = KMeans(n_clusters=20)
kmeans_fashion.fit(X_train_fashion_flat)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_fashion = kmeans_objective(X_train_fashion_flat, kmeans_fashion.centroids, kmeans_fashion.labels)
print(f"KMeans Objective for FASHION (K=20): {objective_fashion}")

purity_fashion = purity_score(y_train_fashion, kmeans_fashion.labels)
gini_fashion = gini_index(y_train_fashion, kmeans_fashion.labels)

print(f"Purity for MNIST (K=20): {purity_fashion}")
print(f"Gini Index for MNIST (K=20): {gini_fashion}")

KMeans Objective for FASHION (K=20): 1609106.6014856338
Purity for MNIST (K=20): 0.6614333333333333
Gini Index for MNIST (K=20): -9.366613925981754


FASHION KMEANS K=5 with external clustering performance.

In [13]:
# Run KMeans
kmeans_fashion = KMeans(n_clusters=5)
kmeans_fashion.fit(X_train_fashion_flat)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_fashion = kmeans_objective(X_train_fashion_flat, kmeans_fashion.centroids, kmeans_fashion.labels)
print(f"KMeans Objective for FASHION (K=5): {objective_fashion}")

purity_fashion = purity_score(y_train_fashion, kmeans_fashion.labels)
gini_fashion = gini_index(y_train_fashion, kmeans_fashion.labels)

print(f"Purity for MNIST (K=5): {purity_fashion}")
print(f"Gini Index for MNIST (K=5): {gini_fashion}")

KMeans Objective for FASHION (K=5): 2352605.6906437874
Purity for MNIST (K=5): 0.41091666666666665
Gini Index for MNIST (K=5): nan


<ipython-input-11-cf48c710b723>:10: RuntimeWarning: invalid value encountered in divide
  cluster_purities = np.amax(confusion_mat, axis=0) / cluster_sizes


C) run KMeans on the 20NG Dataset, try K=20

Load and prepare the data.

In [14]:
# Download the 20 Newsgroups dataset
!wget -O /content/20news-bydate.tar.gz 'http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz'

# Extract the dataset
import tarfile
import os

data_dir = '/content/20news-bydate'
with tarfile.open('/content/20news-bydate.tar.gz', 'r:gz') as tar:
    tar.extractall(path=data_dir)

print(f"Dataset extracted to {data_dir}")

--2025-02-03 22:01:24--  http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz
Resolving qwone.com (qwone.com)... 173.48.205.131
Connecting to qwone.com (qwone.com)|173.48.205.131|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14464277 (14M) [application/x-gzip]
Saving to: ‘/content/20news-bydate.tar.gz’

/content/20news-byd 100%[===================>]  13.79M  5.32MB/s    in 2.6s    

2025-02-03 22:01:26 (5.32 MB/s) - ‘/content/20news-bydate.tar.gz’ saved [14464277/14464277]

Dataset extracted to /content/20news-bydate


Note on TF-IDF Vectorization:
Term Frequency-Inverse Document Frequency is a text representation technique that evaluates how importat a word is to a document relative to a corpus.

Term Frequency (TF) = (word count in doc) / (total words in doc) - measures local importance of a word

Inverse document frequency (IDF) = log(total docs / docs containing word) - penalizes words common across many docs

TF-IDF = TF * IDF - final importance score for each word

TfidVectorizer automatically applies L2 Normalization to our output vectors.


In [15]:
# Parse the dataset
import os
import tarfile
from sklearn.feature_extraction.text import TfidfVectorizer

data_dir = '/content/20news-bydate'

# Check if the directory exists, if not, download and extract
train_dir = os.path.join(data_dir, '20news-bydate-train')
if not os.path.exists(train_dir):
    print("Downloading 20 Newsgroups dataset...")
    !wget -O /content/20news-bydate.tar.gz 'http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz'

    print("Extracting dataset...")
    with tarfile.open('/content/20news-bydate.tar.gz', 'r:gz') as tar:
        tar.extractall(path=data_dir)

    print(f"Dataset extracted to {data_dir}")
def parse_20newsgroups(data_dir):
    texts, labels = [], []
    label_names = sorted(os.listdir(data_dir))

    for label_idx, label_name in enumerate(label_names):
        label_path = os.path.join(data_dir, label_name)
        if os.path.isdir(label_path):
            for file_name in os.listdir(label_path):
                file_path = os.path.join(label_path, file_name)
                with open(file_path, 'r', errors='ignore') as f:
                    texts.append(f.read())
                    labels.append(label_idx)

    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    X = vectorizer.fit_transform(texts).toarray()

    return X, labels, vectorizer.get_feature_names_out()

# Process the dataset
X_20NG, y_20NG, feature_names_20NG = parse_20newsgroups(os.path.join(data_dir, '20news-bydate-train'))
print(f"20NG Dataset: {len(X_20NG)} samples, {len(feature_names_20NG)} features.")

20NG Dataset: 11314 samples, 5000 features.


Now with 20 NewsGroups loaded we can run our KMEANS with K = 20.

In [16]:
# Run KMeans
kmeans_20NG = KMeans(n_clusters=20)
kmeans_20NG.fit(X_20NG)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_20NG = kmeans_objective(X_20NG, kmeans_20NG.centroids, kmeans_20NG.labels)
print(f"KMeans Objective for 20NG (K=20): {objective_20NG}")

KMeans Objective for 20NG (K=20): 10588.172849674344


In [17]:
from sklearn.metrics import confusion_matrix

def purity_score(y_true, y_pred):
    confusion_mat = confusion_matrix(y_true, y_pred)
    return np.sum(np.amax(confusion_mat, axis=0)) / np.sum(confusion_mat)

def gini_index(y_true, y_pred):
    contingency = confusion_matrix(y_true, y_pred)
    cluster_sizes = np.sum(contingency, axis=0)

    # Handle empty clusters and division by zero
    valid_clusters = cluster_sizes > 0
    if np.sum(valid_clusters) == 0:
        return 1.0  # Return worst case when all clusters are empty

    # Calculate purities only for valid clusters
    cluster_purities = np.zeros_like(cluster_sizes, dtype=np.float64)
    cluster_purities[valid_clusters] = np.amax(contingency[:, valid_clusters], axis=0) / cluster_sizes[valid_clusters]

    # Calculate weighted Gini index (proper formula)
    total_samples = np.sum(cluster_sizes)
    weights = cluster_sizes[valid_clusters] / total_samples
    gini = 1 - np.sum(weights * (cluster_purities[valid_clusters] ** 2))

    return np.clip(gini, 0.0, 1.0)  # Ensure valid range

purity_20NG = purity_score(y_20NG, kmeans_20NG.labels)
gini_20NG = gini_index(y_20NG, kmeans_20NG.labels)

print(f"Purity for MNIST (K=20): {purity_20NG}")
print(f"Gini Index for MNIST (K=20): {gini_20NG}")

Purity for MNIST (K=20): 0.3485062754109952
Gini Index for MNIST (K=20): -6.627868808990032


20NG KMEANS with K = 40 with external clustering performance.

In [18]:
# Run KMeans
kmeans_20NG = KMeans(n_clusters=40)
kmeans_20NG.fit(X_20NG)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_20NG = kmeans_objective(X_20NG, kmeans_20NG.centroids, kmeans_20NG.labels)
print(f"KMeans Objective for 20NG (K=40): {objective_20NG}")

purity_20NG = purity_score(y_20NG, kmeans_20NG.labels)
gini_20NG = gini_index(y_20NG, kmeans_20NG.labels)

print(f"Purity for MNIST (K=40): {purity_20NG}")
print(f"Gini Index for MNIST (K=40): {gini_20NG}")

KMeans Objective for 20NG (K=40): 10324.724013294199
Purity for MNIST (K=40): 0.40215662011666964
Gini Index for MNIST (K=40): -12.106054976888613


20NG KMEANS with K = 10 with external clustering performance.

In [19]:
# Run KMeans
kmeans_20NG = KMeans(n_clusters=10)
kmeans_20NG.fit(X_20NG)

# Evaluate the KMeans objective
def kmeans_objective(X, centroids, labels):
    return np.sum([np.linalg.norm(X[labels == k] - centroids[k])**2 for k in range(centroids.shape[0])])

objective_20NG = kmeans_objective(X_20NG, kmeans_20NG.centroids, kmeans_20NG.labels)
print(f"KMeans Objective for 20NG (K=10): {objective_20NG}")

purity_20NG = purity_score(y_20NG, kmeans_20NG.labels)
gini_20NG = gini_index(y_20NG, kmeans_20NG.labels)

print(f"Purity for MNIST (K=10): {purity_20NG}")
print(f"Gini Index for MNIST (K=10): {gini_20NG}")

KMeans Objective for 20NG (K=10): 10766.72516519991
Purity for MNIST (K=10): 0.32587944140003533
Gini Index for MNIST (K=10): nan


<ipython-input-17-229c53d8b175>:10: RuntimeWarning: invalid value encountered in divide
  cluster_purities = np.amax(confusion_mat, axis=0) / cluster_sizes


D) run soft Kmeans for MNIST dataset, use K=10. You can try beta=0.1, beta=1, beta=10. Evaluate performance.

In [21]:
import numpy as np
from sklearn.metrics import confusion_matrix

class SoftKMeans:
    def __init__(self, n_clusters=10, max_iter=100, tol=1e-4, beta=1.0):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.beta = beta
        self.centroids = None
        self.responsibilities = None

    def _compute_distances(self, X):
        """Calculate Euclidean distances between data points and centroids"""
        return np.linalg.norm(X[:, np.newaxis] - self.centroids, axis=2)

    def _compute_responsibilities(self, distances):
        """Convert distances to probabilities using softmax with temperature parameter beta"""
        scaled_dist = -self.beta * distances
        shifted_scaled = scaled_dist - np.max(scaled_dist, axis=1, keepdims=True)  # For numerical stability
        exp_values = np.exp(shifted_scaled)
        return exp_values / np.sum(exp_values, axis=1, keepdims=True)

    def fit(self, X):
        # Initialize centroids randomly
        self.centroids = X[np.random.choice(X.shape[0], self.n_clusters, replace=False)]

        for _ in range(self.max_iter):
            # E-step: Compute responsibilities
            distances = self._compute_distances(X)
            responsibilities = self._compute_responsibilities(distances)

            # M-step: Update centroids
            new_centroids = np.zeros_like(self.centroids)
            for k in range(self.n_clusters):
                weight_sum = np.sum(responsibilities[:, k])
                if weight_sum < 1e-8:  # Handle empty clusters more robustly
                    new_centroids[k] = X[np.random.choice(X.shape[0])]
                else:
                    new_centroids[k] = np.sum(X * responsibilities[:, k, np.newaxis], axis=0) / weight_sum

            # Check convergence
            if np.linalg.norm(new_centroids - self.centroids) < self.tol:
                break
            self.centroids = new_centroids

        # Store final responsibilities
        self.responsibilities = self._compute_responsibilities(self._compute_distances(X))

    def predict(self, X):
        """Assign to cluster with highest responsibility"""
        distances = self._compute_distances(X)
        return np.argmin(distances, axis=1)

def purity_score(y_true, y_pred):
    contingency = confusion_matrix(y_true, y_pred)
    return np.sum(np.amax(contingency, axis=0)) / np.sum(contingency)

def gini_index(y_true, y_pred):
    contingency = confusion_matrix(y_true, y_pred)
    cluster_sizes = np.sum(contingency, axis=0)

    # Handle empty clusters and division by zero
    valid_clusters = cluster_sizes > 0
    if np.sum(valid_clusters) == 0:
        return 1.0  # Worst possible case when all clusters are empty

    # Calculate purities only for valid clusters
    cluster_purities = np.zeros_like(cluster_sizes, dtype=np.float64)
    cluster_purities[valid_clusters] = np.amax(contingency[:, valid_clusters], axis=0) / cluster_sizes[valid_clusters]

    # Calculate weighted Gini index
    total_samples = np.sum(cluster_sizes)
    weights = cluster_sizes[valid_clusters] / total_samples
    gini = 1 - np.sum(weights * (cluster_purities[valid_clusters] ** 2))

    return np.clip(gini, 0.0, 1.0)  # Ensure valid range

# Run experiments with different beta values
betas = [0.1, 1, 10]
for beta in betas:
    print(f"\nRunning Soft K-Means with β={beta}")
    skm = SoftKMeans(n_clusters=10, beta=beta, max_iter=100)
    skm.fit(X_train_mnist_flat)

    # Get cluster assignments
    cluster_labels = skm.predict(X_train_mnist_flat)

    # Calculate metrics
    purity = purity_score(y_train_mnist, cluster_labels)
    gini = gini_index(y_train_mnist, cluster_labels)

    print(f"β={beta}:")
    print(f"Purity = {purity:.4f}")
    print(f"Gini Index = {gini:.4f}")


Running Soft K-Means with β=0.1
β=0.1:
Purity = 0.3594
Gini Index = 0.8671

Running Soft K-Means with β=1
β=1:
Purity = 0.2096
Gini Index = 0.9560

Running Soft K-Means with β=10
β=10:
Purity = 0.5871
Gini Index = 0.6100
